# ***Créditos - Distribución***

In [2]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

## *Data-import*

In [3]:
# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, 'datos_json')
#print("Directorio JSON relativo:", directorio_json)
# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

In [4]:
dataframes = {} # Crear un diccionario para almacenar los DataFrames
# Iterar sobre cada archivo JSON y crear un DataFrame
for archivo in archivos_json:
    # Obtener el nombre de la tabla del nombre del archivo
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')    
    # Cargar el archivo JSON en un DataFrame y asignarlo a una variable con un nombre dinámico
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

In [5]:
# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())
# Filtrar los nombres para obtener solo aquellos que comienzan con "df_"
nombres_df = [nombre for nombre in nombres_variables_globales if nombre.startswith("df_")]

# Imprimir la lista de nombres de los DataFrames creados
print("Lista de DataFrames creados:")
print(nombres_df)

Lista de DataFrames creados:
['df_alfa_q_feb_2023_pachuca', 'df_alfa_q_jul_2023_pachuca', 'df_alfa_q_jun_2023_pachuca', 'df_alfa_q_mar_2023_pachuca', 'df_alfa_q_nov_2022_pachuca', 'df_alfa_q_oct_2022_pachuca', 'df_alfa_q_sep_2023_pachuca', 'df_enero_2024_querertaro', 'df_financiamientos_2022_pachuca', 'df_financiamientos_2023_pachuca', 'df_grupos_edad_pachuca', 'df_grupos_edad_queretaro', 'df_junio_2023_queretaro', 'df_salarios_pachuca', 'df_usuarios']


___
___
# **1** *Distribución de créditos por segmento*

### 2022

In [5]:
dist_financiamientos_2022_pachuca = df_financiamientos_2022_pachuca[['id','monto']]
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1600000.00),
    'C': (1600001.00, 2300000.00),
    'B': (2300001.00, 3000000.00),
    'A': (3000001.00, 3900000.00),
    'S': (3900000.00, 20000000.00)
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
dist_financiamientos_2022_pachuca['segmento'] = dist_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)
dist_financiamientos_2022_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_21248\1187833169.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dist_financiamientos_2022_pachuca['segmento'] = dist_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)


,id,monto,segmento
0,1,0.00,E
1,2,1563551.33,D
2,3,1263770.32,D
3,4,3614324.42,A
4,5,252907.18,E


In [6]:
# Calcular cantidad de registros por segmento
creditos_por_segmento = dist_financiamientos_2022_pachuca['segmento'].value_counts()
# Calcular el porcentaje de cada segmento
porcentaje_por_segmento = (creditos_por_segmento / len(dist_financiamientos_2022_pachuca)) * 100

# Crear DataFrame con los resultados
creditos_segmentos_2022_pachuca = pd.DataFrame({
    'Segmento': creditos_por_segmento.index,
    'Cantidad': creditos_por_segmento.values,
    'Porcentaje': porcentaje_por_segmento.values
})

# Ordenar por segmento
creditos_segmentos_2022_pachuca.sort_values(by='Segmento', inplace=True)
creditos_segmentos_2022_pachuca

,Segmento,Cantidad,Porcentaje
5,A,54,3.890490
4,B,85,6.123919
2,C,149,10.734870
1,D,236,17.002882
0,E,758,54.610951
3,S,106,7.636888


In [7]:
colores = ['#3d45c0','#ff4853','#ffb038','#7cc668','#67c2df','#6773df']
porcentajes = creditos_segmentos_2022_pachuca['Porcentaje']
labels = creditos_segmentos_2022_pachuca['Segmento']

fig = go.Figure(data=[go.Pie(labels=labels, values=porcentajes, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent', textposition='inside')
fig.update_layout(
    title='Distribución de créditos por segmento 2022', # Agregar título
    #showlegend=False  # Ocultar leyenda
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_pie_discreditossegmento_2022_pachuca', carpeta='graficas')

fig.show()

### 2023

In [8]:
dist_financiamientos_2023_pachuca = df_financiamientos_2023_pachuca[['id','monto']]
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1600000.00),
    'C': (1600001.00, 2300000.00),
    'B': (2300001.00, 3000000.00),
    'A': (3000001.00, 3900000.00),
    'S': (3900000.00, 20000000.00)
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
dist_financiamientos_2023_pachuca['segmento'] = dist_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
dist_financiamientos_2023_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_21248\2798921774.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,segmento
0,1,0.0,E
1,2,0.0,E
2,3,0.0,E
3,4,0.0,E
4,5,0.0,E


In [9]:
# Calcular cantidad de registros por segmento
creditos_por_segmento = dist_financiamientos_2023_pachuca['segmento'].value_counts()
# Calcular el porcentaje de cada segmento
porcentaje_por_segmento = (creditos_por_segmento / len(dist_financiamientos_2023_pachuca)) * 100

# Crear DataFrame con los resultados
creditos_segmentos_2023_pachuca = pd.DataFrame({
    'Segmento': creditos_por_segmento.index,
    'Cantidad': creditos_por_segmento.values,
    'Porcentaje': porcentaje_por_segmento.values
})

# Ordenar por segmento
creditos_segmentos_2023_pachuca.sort_values(by='Segmento', inplace=True)
creditos_segmentos_2023_pachuca

,Segmento,Cantidad,Porcentaje
5,A,19,2.790015
3,B,43,6.314244
2,C,63,9.251101
1,D,127,18.649046
0,E,386,56.681351
4,S,43,6.314244


In [10]:
colores = ['#3d45c0','#ff4853','#ffb038','#7cc668','#67c2df','#6773df']
porcentajes = creditos_segmentos_2023_pachuca['Porcentaje']
labels = creditos_segmentos_2023_pachuca['Segmento']

fig = go.Figure(data=[go.Pie(labels=labels, values=porcentajes, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent', textposition='inside')
fig.update_layout(
    title='Distribución de créditos por segmento 2023', # Agregar título
    #showlegend=False  # Ocultar leyenda
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_pie_discreditossegmento_2023_pachuca', carpeta='graficas')

fig.show()

___
___
# **2** *Distribución % de créditos por segmento (trimestre)*

In [11]:
trimestre_financiamientos_2022_pachuca = df_financiamientos_2022_pachuca[['id','mes','monto']]
trimestre_financiamientos_2023_pachuca = df_financiamientos_2023_pachuca[['id','mes','monto']]

### 2022
#### *Asignamos trimestres*

In [12]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
trimestre_financiamientos_2022_pachuca['trimestre'] = trimestre_financiamientos_2022_pachuca['mes'].apply(asignar_trimestre)
trimestre_financiamientos_2022_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_21248\3798387835.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre
0,1,1,0.00,1
1,2,1,1563551.33,1
2,3,1,1263770.32,1
3,4,1,3614324.42,1
4,5,1,252907.18,1
...,...,...,...,...
1383,1384,12,0.00,4
1384,1385,12,0.00,4
1385,1386,12,0.00,4
1386,1387,12,0.00,4


In [13]:
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1750000.00),
    'C': (1750001.00, 2500000.00),
    'B': (2500001.00, 3250000.00),
    'A': (3250001.00, 4000000.00),
    'S': (4000001.00, float('inf'))  # Segmento S con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
trimestre_financiamientos_2022_pachuca['segmento'] = trimestre_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)
trimestre_financiamientos_2022_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_21248\750050574.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre,segmento
0,1,1,0.00,1,E
1,2,1,1563551.33,1,D
2,3,1,1263770.32,1,D
3,4,1,3614324.42,1,A
4,5,1,252907.18,1,E


In [14]:
trimestres_credit_2022_pachuca = trimestre_financiamientos_2022_pachuca[['trimestre','segmento']]
trimestres_credit_2022_pachuca

,trimestre,segmento
0,1,E
1,1,D
2,1,D
3,1,A
4,1,E
...,...,...
1383,4,E
1384,4,E
1385,4,E
1386,4,E


In [15]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2022_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2022_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  1388
trimestre          1          2          3          4
segmento                                             
A           0.576369   0.504323   0.720461   0.720461
B           1.080692   1.512968   1.080692   1.585014
C           2.953890   2.521614   2.233429   2.593660
D           4.610951   5.043228   5.475504   4.682997
E          13.112392  12.103746  12.752161  16.642651
S           1.368876   1.585014   2.305476   2.233429


In [16]:
colores = ['#3d45c0','#ff4853','#ffb038','#7cc668']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'Trimestre {trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=porcentajes,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))
fig.update_layout(
    #barmode='stack', 
    title='Distribución de créditos por segmento (trimestre) 2022',
    yaxis=dict(
        #tickformat='2.2%',
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditossegmento_trimestre2022_pachuca', carpeta='graficas')

fig.show()


### 2023
#### *Asignamos trimestres*

In [17]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
trimestre_financiamientos_2023_pachuca['trimestre'] = trimestre_financiamientos_2023_pachuca['mes'].apply(asignar_trimestre)
trimestre_financiamientos_2023_pachuca

C:\Users\yoe11\AppData\Local\Temp\ipykernel_21248\1639858340.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre
0,1,1,0.00,1
1,2,1,0.00,1
2,3,1,0.00,1
3,4,1,0.00,1
4,5,1,0.00,1
...,...,...,...,...
676,677,7,4817952.41,3
677,678,7,5949855.28,3
678,679,7,8000000.00,3
679,680,7,9650850.00,3


In [18]:
# Definir los rangos de monto de cada segmento
segmentos = {
    'E': (0.00, 1000000.00),
    'D': (1000001.00, 1750000.00),
    'C': (1750001.00, 2500000.00),
    'B': (2500001.00, 3250000.00),
    'A': (3250001.00, 4000000.00),
    'S': (4000001.00, float('inf'))  # Segmento S con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
trimestre_financiamientos_2023_pachuca['segmento'] = trimestre_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
trimestre_financiamientos_2023_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_21248\4014434308.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,mes,monto,trimestre,segmento
0,1,1,0.0,1,E
1,2,1,0.0,1,E
2,3,1,0.0,1,E
3,4,1,0.0,1,E
4,5,1,0.0,1,E


In [19]:
trimestres_credit_2023_pachuca = trimestre_financiamientos_2023_pachuca[['trimestre','segmento']]
trimestres_credit_2023_pachuca

,trimestre,segmento
0,1,E
1,1,E
2,1,E
3,1,E
4,1,E
...,...,...
676,3,S
677,3,S
678,3,S
679,3,S


In [20]:
# Calcular el total de registros
total_registros = len(trimestres_credit_2023_pachuca)
print("Total de registros: ", total_registros)
# Calcular la tabla de frecuencias relativas
tabla_frecuencias = pd.pivot_table(trimestres_credit_2023_pachuca, 
                                   index='segmento', 
                                   columns='trimestre', 
                                   aggfunc='size', 
                                   fill_value=0)

# Convertir los valores a porcentajes
tabla_porcentajes = tabla_frecuencias / total_registros * 100
print(tabla_porcentajes)

Total de registros:  681
trimestre          1          2         3
segmento                                 
A           0.881057   1.027900  0.146843
B           1.762115   1.908957  1.468429
C           4.552129   2.790015  1.027900
D           7.929515   9.397944  4.258443
E          21.292217  28.046990  7.342144
S           2.936858   2.349486  0.881057


In [21]:
colores = ['#3d45c0','#ff4853','#ffb038','#7cc668']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
for i, trimestre in enumerate(trimestres):
    fig.add_trace(go.Bar(
        name=f'Trimestre {trimestre}', 
        x=tabla_porcentajes.index, 
        y=tabla_porcentajes[trimestre], 
        marker_color=colores[i],
        text=porcentajes,  # Texto que se mostrará en las barras (porcentaje)
        textposition='outside',  # Posición del texto (puede ser 'inside' o 'outside')
        texttemplate='%{text:.1f}%', 
    ))
fig.update_layout(
    #barmode='stack', 
    title='Distribución de créditos por segmento (trimestre) 2023',
    yaxis=dict(
        #tickformat='2.2%',
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditossegmento_trimestre2023_pachuca', carpeta='graficas')

fig.show()


___
___
# **3** *Distribución de créditos por monto*

In [22]:
monto_financiamientos_2022_pachuca = df_financiamientos_2022_pachuca[['id','monto']]
monto_financiamientos_2023_pachuca = df_financiamientos_2023_pachuca[['id','monto']]

In [23]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
monto_financiamientos_2022_pachuca['rango'] = monto_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)
monto_financiamientos_2022_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_21248\4270173908.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,rango
0,1,0.00,1
1,2,1563551.33,2
2,3,1263770.32,2
3,4,3614324.42,4
4,5,252907.18,1


In [24]:
conteo_por_rango = monto_financiamientos_2022_pachuca['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango

,rango,conteo
0,1,758
1,2,333
2,3,137
3,5,104
4,4,56


In [25]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Reemplazar los valores de rango con el texto correspondiente utilizando replace
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace({
    '1': '$0.00 - $1,000,000.00',
    '2': '$1,000,001.00 - $2,000,000.00',
    '3': '$2,000,001.00 - $3,000,000.00',
    '4': '$3,000,001.00 - $4,000,000.00',
    '5': '$4,000,001.00 - $20,000,000.00'    
})
df_conteo_por_rango

,rango,conteo
0,"$0.00 - $1,000,000.00",758
1,"$1,000,001.00 - $2,000,000.00",333
2,"$2,000,001.00 - $3,000,000.00",137
3,"$4,000,001.00 - $20,000,000.00",104
4,"$3,000,001.00 - $4,000,000.00",56


In [26]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']
colores = ['#3d45c0','#ff4853','#ffb038','#7cc668','#67c2df','#6773df']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    title='Distribución de créditos por monto 2022',
    #title_font_size = 20,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditosmonto_2022_pachuca', carpeta='graficas')

fig.show()


In [27]:
# Definir los rangos de los monto
segmentos = {
    '1': (0.00, 1000000.00),
    '2': (1000001.00, 2000000.00),
    '3': (2000001.00, 3000000.00),
    '4': (3000001.00, 4000000.00),
    '5': (4000001.00, float('inf'))  # Rango con límite superior infinito
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
monto_financiamientos_2023_pachuca['rango'] = monto_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
monto_financiamientos_2023_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_21248\1660705631.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,rango
0,1,0.0,1
1,2,0.0,1
2,3,0.0,1
3,4,0.0,1
4,5,0.0,1


In [28]:
conteo_por_rango = monto_financiamientos_2023_pachuca['rango'].value_counts()
df_conteo_por_rango = pd.DataFrame({
    'rango': conteo_por_rango.index,
    'conteo': conteo_por_rango.values
})
# Nuevo registro que queremos agregar
#nuevo_registro = {'rango': 5, 'conteo': 0}
# Agregar el nuevo registro al DataFrame
#df_conteo_por_rango = df_conteo_por_rango.append(nuevo_registro, ignore_index=True)
df_conteo_por_rango

,rango,conteo
0,1,386
1,2,167
2,3,66
3,5,42
4,4,20


In [29]:
# Convertir la columna 'rango' a cadena de texto (str) antes de realizar el reemplazo
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].astype(str)

# Reemplazar los valores de rango con el texto correspondiente utilizando replace
df_conteo_por_rango['rango'] = df_conteo_por_rango['rango'].replace({
    '1': '$0.00 - $1,000,000.00',
    '2': '$1,000,001.00 - $2,000,000.00',
    '3': '$2,000,001.00 - $3,000,000.00',
    '4': '$3,000,001.00 - $4,000,000.00',
    '5': '$4,000,001.00 - $20,000,000.00'    
})
df_conteo_por_rango

,rango,conteo
0,"$0.00 - $1,000,000.00",386
1,"$1,000,001.00 - $2,000,000.00",167
2,"$2,000,001.00 - $3,000,000.00",66
3,"$4,000,001.00 - $20,000,000.00",42
4,"$3,000,001.00 - $4,000,000.00",20


In [30]:
labels = df_conteo_por_rango['rango']
values = df_conteo_por_rango['conteo']
colores = ['#3d45c0','#ff4853','#ffb038','#7cc668','#67c2df','#6773df']

fig = go.Figure()
trimestres = tabla_porcentajes.columns
fig.add_trace(go.Bar(
        name=f'Trimestre {trimestre}', 
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,  # Texto que se mostrará en las barras (porcentaje)
        textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
        #texttemplate='%{text:.1f}%', 
    )
)    
fig.update_layout(
    #barmode='stack', 
    title='Distribución de créditos por monto 2023',
    #title_font_size = 22,
    yaxis=dict(
        #title='M', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y         
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_bar_discreditosmonto_2023_pachuca', carpeta='graficas')

fig.show()


___
___
# **4** *Promedio de valor de Créditos para casas nueva y para vivienda usada*

___
___
# **5** *Estacionalidad*

In [17]:
finance_2022_pachuca = df_financiamientos_2022_pachuca[['id','año','mes']]
finance_2023_pachuca = df_financiamientos_2023_pachuca[['id','año','mes']]

estacionalidad_2223_pachuca = pd.concat([finance_2022_pachuca, finance_2023_pachuca], ignore_index=True)
estacionalidad_2223_pachuca

,id,año,mes
0,1,2022,1
1,2,2022,1
2,3,2022,1
3,4,2022,1
4,5,2022,1
...,...,...,...
2064,677,2023,7
2065,678,2023,7
2066,679,2023,7
2067,680,2023,7


In [18]:
# Definir una función para asignar el trimestre según el número de mes
def asignar_trimestre(mes):
    if mes in range(1, 4):
        return 1
    elif mes in range(4, 7):
        return 2
    elif mes in range(7, 10):
        return 3
    else:
        return 4

# Aplicar la función para asignar el trimestre a cada fila
estacionalidad_2223_pachuca['trimestre'] = estacionalidad_2223_pachuca['mes'].apply(asignar_trimestre)
estacionalidad_2223_pachuca

,id,año,mes,trimestre
0,1,2022,1,1
1,2,2022,1,1
2,3,2022,1,1
3,4,2022,1,1
4,5,2022,1,1
...,...,...,...,...
2064,677,2023,7,3
2065,678,2023,7,3
2066,679,2023,7,3
2067,680,2023,7,3


In [27]:
conteo_trimestral = estacionalidad_2223_pachuca.groupby(['año', 'trimestre']).size().reset_index(name='conteo')
conteo_trimestral = conteo_trimestral[conteo_trimestral['año'] != 0]
# Concatenar 'año' y 'trimestre' en una nueva columna llamada 'trimestre'
conteo_trimestral['trimestre'] = conteo_trimestral['trimestre'].astype(str) + 'o ' + conteo_trimestral['año'].astype(str) 
# Eliminar la columna 'año' si ya no es necesaria
conteo_trimestral.drop(columns=['año'], inplace=True)
conteo_trimestral

,trimestre,conteo
1,1o 2022,329
2,2o 2022,323
3,3o 2022,341
4,4o 2022,395
5,1o 2023,267
6,2o 2023,310
7,3o 2023,103


In [32]:
colores = ['#3d45c0','#ff4853','#ffb038','#7cc668','#67c2df','#6773df','#939fad']

import numpy as np
# Calcula el máximo valor de la serie y redondea hacia arriba al múltiplo más cercano de 50
max_y = np.ceil(conteo_trimestral['conteo'].max() / 50) * 50
# Crea un rango de valores de y en incrementos de 50 hasta el máximo
values_range = np.arange(0, max_y + 1, 50)
# Modifica los valores de 'values' en tu gráfico para que estén en incrementos de 50
values = conteo_trimestral['conteo'].apply(lambda x: min(values_range, key=lambda y: abs(y - x)))
# Luego, puedes proceder con la creación de tu gráfico como lo hiciste antes
labels = conteo_trimestral['trimestre']

fig = go.Figure()
fig.add_trace(go.Bar(
        x=labels, 
        y=values, 
        marker_color=colores,
        text=values,
        textposition='inside',
        showlegend=False,
        #width=0.5 
    )
)    
fig.update_layout(
    title='Número de créditos por trimestre (2022-2023)',
    yaxis=dict(
        gridcolor='#dddcda',
        tickvals=values_range,  # Especifica los valores de los ticks del eje y
        tickmode='array'  # Usa los valores especificados en 'tickvals' para los ticks
    ),  
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(
        yanchor="top",
        y=0.99,
    )
)

guardar_grafico_como_html(fig, 'g_bar_estacionalidad_pachuca', carpeta='graficas')

fig.show()
